In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datetime import datetime

%matplotlib inline
matplotlib.style.use('bmh')

## Assumptions

To do:
* fuel cost per litre
* inflation index (to adjust costs)
* tyre cost (each, installed)
* tyre life (km)
* fuel consumption of car

In [ ]:
km_per_year = 15000

## Standing Costs

* yearly insurance
* registration
* servicing (cost per service)
* service interval (with defaults for second-hand cars)
* insurance
* roadside assist

## Depreciation functions

In [ ]:
def depreciation_rate_for_age(age_in_years):
    '''Go with the simple assumption of 15% for first 3 years, and 10% after that.
       This might be too aggressive, but can fine-tune later.
    '''
    if age_in_years < 3:
        return 0.15
    else:
        return 0.10

### Calculate Depreciation

In [ ]:
purchase_price = 35000
year_of_purchase = datetime.now().year
age_at_purchase = year_of_purchase - 2017
num_years = 15

In [ ]:
dep_value = np.zeros(num_years)
dep_value[0] = purchase_price
previous = dep_value[0]
for yr in range(1, num_years):
    previous *= (1.0 - depreciation_rate_for_age(age_at_purchase + yr))
    dep_value[yr] = previous

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(dep_value);

In [ ]:
dep_value